In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
wv  = torch.load("./dataset/word2vec_emb_tensor.pkl")

In [3]:
cnn = torch.load("./dataset/cnn_emb_tensor.pkl")

In [4]:
X = torch.cat((wv,cnn),dim=1)

In [5]:
df = pd.read_csv("./dataset/books_with_genres.csv")

In [6]:
df.columns

Index(['isbn', 'text_reviews_count', 'series', 'country_code', 'language_code',
       'popular_shelves', 'asin', 'is_ebook', 'average_rating', 'kindle_asin',
       'similar_books', 'description', 'format', 'link', 'authors',
       'publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'edition_information', 'publication_year', 'url',
       'image_url', 'book_id', 'ratings_count', 'work_id', 'title',
       'title_without_series', 'history, historical fiction, biography',
       'fiction', 'fantasy, paranormal', 'mystery, thriller, crime', 'poetry',
       'romance', 'non-fiction', 'children', 'young-adult', 'comics, graphic'],
      dtype='object')

In [7]:
y = df.loc[:,"history, historical fiction, biography" : "comics, graphic"]

In [8]:
y = y.to_numpy()
y = torch.from_numpy(y)

In [9]:
X_train = X[[i for i in range(4000)]]
X_test = X[[i for i in range(4000,5000)]]
y_train = y[[i for i in range(4000)]]
y_test = y[[i for i in range(4000, 5000)]]

In [10]:
y_train=y_train.float()
y_test=y_test.float()

In [11]:
device = torch.device("cuda")

In [12]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu1 = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [13]:
net = MLP(1100,10,512)
net = net.to(device)

In [14]:
criterion = nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam( net.parameters() , lr=5e-4 )

In [15]:
import torch.optim as optim
from random import randint

bs = 20

for epoch in range(50):

    for iter in range(1,500):

        # create a minibatch
        indices=torch.LongTensor(bs).random_(0,4000)
        minibatch_data =  X_train[indices].to(device)
        minibatch_label = y_train[indices].to(device)

        #reshape them to fit the network
        inputs=minibatch_data.view(bs,1100)
        # feed the input to the net  
        inputs.requires_grad_()
        prob=net(inputs)


        # update the weights (all the magic happens here -- we will discuss it later)
        loss = criterion(prob, minibatch_label)    
        optimizer.zero_grad()       
        loss.backward()

In [16]:

running_error=0
num_batches=0


for i in range(0,1000,bs):

    # extract the minibatch
    minibatch_data =  X_test[i:i+bs].to(device)
    minibatch_label= y_test[i:i+bs].to(device)

    # reshape the minibatch
    inputs = minibatch_data.view(bs,1100)

    # feed it to the network
    scores=net( inputs ) 

    # compute the error made on this batch
    error = criterion( scores , minibatch_label)
    
    # add it to the running error
    running_error += error.item()

    num_batches+=1


# compute error rate on the full test set
total_error = running_error/num_batches

print( 'error rate on test set =', total_error*100 ,'percent')

error rate on test set = 69.70403611660004 percent
